In [1]:
import sys
sys.path.extend(["/home/mhasan3/Desktop/WorkFolder/"])

In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os, glob

In [3]:
args = {
    "models" : r"/home/mhasan3/Desktop/WorkFolder/keras_dir/biblio2/chap6_ensembling/models/"
}

In [4]:
testx, testy = cifar10.load_data()[1]
testx = testx.astype('float') / 255.0

# init the labels names
unique_class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog",
                "frog", "horse", "ship", "truck"]

# encode the labels
lb = LabelBinarizer()
testy = lb.fit_transform(testy)

In [5]:
# construct the path to models
model_paths = os.path.join(args["models"], "*.hdf5")
model_paths = list(glob.glob(model_paths))
models = []

# loop over model paths
for i, model_path in enumerate(model_paths):
    print(f'[INFO] loading model {i+1}/{len(model_paths)}.....')
    models.append(load_model(model_path))

[INFO] loading model 1/5.....
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
[INFO] loading model 2/5.....
[INFO] loading model 3/5.....
[INFO] loading model 4/5.....
[INFO] loading model 5/5.....


In [6]:
# initialise the list of predictions
print(f'[INFO] evaluating the ensembles.....')
preds = []

# loop over the models
model : Model
for model in models:
    preds.append(model.predict(testx, batch_size=64))

preds = np.average(preds, axis=0)

# evaluate the network
print (classification_report(testy.argmax(axis=1 ),
preds.argmax(axis=1 ), target_names=unique_class_names))

[INFO] evaluating the ensembles.....
              precision    recall  f1-score   support

    airplane       0.85      0.93      0.88      1000
  automobile       0.92      0.96      0.94      1000
        bird       0.85      0.81      0.83      1000
         cat       0.81      0.72      0.76      1000
        deer       0.90      0.82      0.86      1000
         dog       0.93      0.69      0.80      1000
        frog       0.75      0.97      0.85      1000
       horse       0.90      0.93      0.92      1000
        ship       0.93      0.94      0.93      1000
       truck       0.92      0.92      0.92      1000

    accuracy                           0.87     10000
   macro avg       0.88      0.87      0.87     10000
weighted avg       0.88      0.87      0.87     10000

